In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-price-prediction/CarPrice_Assignment.csv
/kaggle/input/car-price-prediction/Data Dictionary - carprices.xlsx


## Load Data

In [2]:
df = pd.read_csv("/kaggle/input/car-price-prediction/CarPrice_Assignment.csv")
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

## Missing Values

In [4]:
df.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

## Categorical columns and the value counts

In [5]:
cat_cols = df.select_dtypes("O").columns

In [6]:
for col in cat_cols:
    print(df[col].value_counts())
    print("======================")

CarName
toyota corona           6
toyota corolla          6
peugeot 504             6
subaru dl               4
mitsubishi mirage g4    3
                       ..
mazda glc 4             1
mazda rx2 coupe         1
maxda glc deluxe        1
maxda rx3               1
volvo 246               1
Name: count, Length: 147, dtype: int64
fueltype
gas       185
diesel     20
Name: count, dtype: int64
aspiration
std      168
turbo     37
Name: count, dtype: int64
doornumber
four    115
two      90
Name: count, dtype: int64
carbody
sedan          96
hatchback      70
wagon          25
hardtop         8
convertible     6
Name: count, dtype: int64
drivewheel
fwd    120
rwd     76
4wd      9
Name: count, dtype: int64
enginelocation
front    202
rear       3
Name: count, dtype: int64
enginetype
ohc      148
ohcf      15
ohcv      13
dohc      12
l         12
rotor      4
dohcv      1
Name: count, dtype: int64
cylindernumber
four      159
six        24
five       11
eight       5
two         4
three 

## Train test split of dataset

In [7]:
from sklearn.model_selection import train_test_split
X = df.drop("price",axis = 1)
y = df["price"]
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

## K-fold Traget encoding

In [8]:
from sklearn.model_selection import KFold
import pandas as pd

def k_fold_target_encoding(train_data, test_data, cat_columns, target, n_folds=5):
    train_data = train_data.copy()
    test_data = test_data.copy()
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for col in cat_columns:
        # Initialize encoded column
        train_encoded = pd.Series(index=train_data.index, dtype=float)
        
        # K-Fold Target Encoding for Training Set
        for train_idx, val_idx in kf.split(train_data):
            train_fold, val_fold = train_data.iloc[train_idx], train_data.iloc[val_idx]
            mean_target = train_fold.groupby(col)[target].mean()
            train_encoded.iloc[val_idx] = val_fold[col].map(mean_target)
        
        # Apply encoding to training data
        train_data[col + '_encoded'] = train_encoded.fillna(train_data[target].mean())
        
        # Apply encoding to test data
        full_train_mean = train_data.groupby(col)[target].mean()
        test_data[col + '_encoded'] = test_data[col].map(full_train_mean).fillna(train_data[target].mean())
    
    return train_data, test_data

## K fold Target Encoding 
train_data, test_data = k_fold_target_encoding(df.iloc[X_train.index], df.iloc[X_test.index], cat_columns=cat_cols, target='price')


In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153 entries, 90 to 102
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   car_ID                  153 non-null    int64  
 1   symboling               153 non-null    int64  
 2   CarName                 153 non-null    object 
 3   fueltype                153 non-null    object 
 4   aspiration              153 non-null    object 
 5   doornumber              153 non-null    object 
 6   carbody                 153 non-null    object 
 7   drivewheel              153 non-null    object 
 8   enginelocation          153 non-null    object 
 9   wheelbase               153 non-null    float64
 10  carlength               153 non-null    float64
 11  carwidth                153 non-null    float64
 12  carheight               153 non-null    float64
 13  curbweight              153 non-null    int64  
 14  enginetype              153 non-null    object

## Drop categorical and target columns along with car_ID as its unique for each record

In [10]:

train_data = train_data.drop(cat_cols, axis = 1)
test_data = test_data.drop(cat_cols,axis = 1)

X_train = train_data.drop(columns = ["price","car_ID"], axis = 1)


In [11]:
X_test = test_data.drop(columns = ["price","car_ID"], axis = 1)

## Check if all columns are numeric

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153 entries, 90 to 102
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   car_ID                  153 non-null    int64  
 1   symboling               153 non-null    int64  
 2   wheelbase               153 non-null    float64
 3   carlength               153 non-null    float64
 4   carwidth                153 non-null    float64
 5   carheight               153 non-null    float64
 6   curbweight              153 non-null    int64  
 7   enginesize              153 non-null    int64  
 8   boreratio               153 non-null    float64
 9   stroke                  153 non-null    float64
 10  compressionratio        153 non-null    float64
 11  horsepower              153 non-null    int64  
 12  peakrpm                 153 non-null    int64  
 13  citympg                 153 non-null    int64  
 14  highwaympg              153 non-null    int64 

## XGBRegressor in predicting the prices

In [13]:
from xgboost import XGBRegressor

## Base Model

In [14]:
xgb = XGBRegressor(random_state = 42)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [15]:

# Predict on the test set
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def model_stats(model = xgb, X_test = X_test, y_test = y_test):
    y_pred = model.predict(X_test)
    
    # Calculate scores
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE is the square root of MSE
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print("Mean Squared Error (MSE):", mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("Mean Absolute Error (MAE):", mae)
    print("R-squared (R2):", r2)

### Performance tuning the model
    1. max_depth
    2. n_estimators
    3. colsample_bylevel
    4. objective: reg:linear
    5. learning_rate
    6. gammea - Minimum reduction in loss at every split(post pruning regularication)

## Hyper Parameter Tuning with GridSearchCV

In [16]:

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

# Parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200],           # Number of boosting rounds
    'learning_rate': [0.01,0.1],          # Step size
    'max_depth': [5, 7],                  # Tree depth
    'colsample_bytree': [0.8, 1.0],       # Feature ratio per tree
}

# Set up GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',    # RMSE as the scoring metric
    cv=5,                                     # 5-fold cross-validation
    verbose=1,
    n_jobs=-1                                # Use all CPU cores
)

# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters and score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Convert back to positive RMSE
print("Best Parameters:", best_params)
print("Best RMSE Score:", best_score)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Best RMSE Score: 2694.920250990446


In [17]:
xgb_tuned = XGBRegressor(colsample_bytree= 0.8, learning_rate= 0.1, max_depth= 5, n_estimators = 100)
xgb_tuned.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [18]:
model_stats(model = xgb)

Mean Squared Error (MSE): 6070784.33722204
Root Mean Squared Error (RMSE): 2463.896170138271
Mean Absolute Error (MAE): 1599.7511454326923
R-squared (R2): 0.910274521271715


In [19]:
model_stats(model = xgb_tuned)

Mean Squared Error (MSE): 5257333.037462859
Root Mean Squared Error (RMSE): 2292.8874890545458
Mean Absolute Error (MAE): 1528.5063756760817
R-squared (R2): 0.9222972358401651


### The tuned model reduced the error metrics and improved r2 score
### Note: XGBRegressor default uses squared error as objective

## Linear regression

In [20]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [21]:
model_stats(model = lin_reg)

Mean Squared Error (MSE): 9284054.436379153
Root Mean Squared Error (RMSE): 3046.974636648483
Mean Absolute Error (MAE): 1941.79641743262
R-squared (R2): 0.8627827670081982
